In [1]:
import sys, os
import numpy as np
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache

# INPUT

In [2]:
inputfilename = './inputs/day12A.txt'

inputdata = read_txt(inputfilename)

testdata = ["RRRRIICCFF",
"RRRRIICCCF",
"VVRRRCCFFF",
"VVRCCCJFFF",
"VVVVCJJCFE",
"VVIVCCJJEE",
"VVIIICJJEE",
"MIIIIIJJEE",
"MIIISIJEEE",
"MMMISSJEEE"]

testdata2 = ["AAAAAA",
"AAABBA",
"AAABBA",
"ABBAAA",
"ABBAAA",
"AAAAAA"]

In [30]:
data_to_use = inputdata

garden_map = np.array([[char for char in line] for line in data_to_use])
garden_map

array([['S', 'S', 'F', ..., 'K', 'K', 'K'],
       ['F', 'F', 'F', ..., 'K', 'K', 'K'],
       ['F', 'F', 'F', ..., 'K', 'K', 'K'],
       ...,
       ['P', 'P', 'P', ..., 'U', 'L', 'L'],
       ['P', 'P', 'P', ..., 'U', 'L', 'L'],
       ['P', 'P', 'P', ..., 'U', 'U', 'U']], shape=(140, 140), dtype='<U1')

## PART 1

In [6]:
hor_ver_movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def get_plant_land_in_garden(garden_map, plant_type):
    plant_map = np.where(garden_map == plant_type, '1', '0')
    plant_list_raw = np.where(plant_map == '1')
    plant_list = set(zip(plant_list_raw[0], plant_list_raw[1]))
    return plant_map, plant_list

def get_all_plant_types_in_garden(garden_map):
    plant_types = set(garden_map.flatten())
    return plant_types

def get_plant_single_region_cost(plant_map, plant_position):
    area = 0
    perimeter = 0
    plant_position_list = set([plant_position])
    plant_positions_visited = set()
    while len(plant_position_list) > 0:
        position = plant_position_list.pop()
        plant_positions_visited.add(position)
        if plant_map[position[0], position[1]] == '1':
            area += 1
            perimeter += 4
            for hor_ver_movement in hor_ver_movements:
                next_position = (position[0] + hor_ver_movement[0], position[1] + hor_ver_movement[1])
                # Invalid positions -- out of the map
                if next_position[0] < 0 or next_position[1] < 0:
                    continue
                try:
                    if plant_map[next_position] == '1':
                        perimeter -= 1
                        if next_position not in plant_positions_visited:
                            plant_position_list.add(next_position)
                except IndexError:
                    pass

    return area*perimeter, plant_positions_visited

def get_plant_total_fence_cost(garden_map, plant_type):
    plant_map, plant_position_list = get_plant_land_in_garden(garden_map, plant_type)
    plant_fence_cost = 0
    while len(plant_position_list) > 0:
        plant_position = plant_position_list.pop()
        added_cost, plant_positions_visited = get_plant_single_region_cost(plant_map, plant_position)
        plant_position_list = plant_position_list.difference(plant_positions_visited)
        plant_fence_cost += added_cost
    return plant_fence_cost

def get_all_plants_total_fence_cost(garden_map):
    plant_fence_cost = 0
    plant_types = get_all_plant_types_in_garden(garden_map)
    for plant_type in plant_types:
        plant_fence_cost += get_plant_total_fence_cost(garden_map, plant_type)
    return plant_fence_cost

In [7]:
# Solve part 1
get_all_plants_total_fence_cost(garden_map)

1184

## PART 2

In [28]:
side_map_tiles = {'left': {'top', 'bottom'}, # Left
             'right': {'top', 'bottom'}, # Right
             'top': {'left', 'right'}, # Top
             'bottom': {'left', 'right'} # Bottom
            }

hor_ver_movements_map = {'right': (0, 1), 'left': (0, -1), 'bottom': (1, 0), 'top': (-1, 0)}

def edge_exists_in_map(plant_map, position, edge_side):
    hor_ver_movement = hor_ver_movements_map[edge_side]
    next_position = (position[0] + hor_ver_movement[0], position[1] + hor_ver_movement[1])
    # Make sure the position is 1
    try:
        assert(plant_map[position]) == '1'
    except:
        # If the position is outside the map or it is 0 there is no edge
        return False
    # Map edge has always an edge
    if next_position[0] < 0 or next_position[1] < 0:
        return True
    try:
        return plant_map[next_position] == '0'
    # Again map edge
    except IndexError:
        return True
    
def add_new_side(plant_map, position, edge_side, visited_positions):
    # Edge is new side UNLESS same edge exists in an already in a visited position aligned with this one
    if edge_exists_in_map(plant_map, position, edge_side):
        for side_map_tile in side_map_tiles[edge_side]:
            hor_ver_movement = hor_ver_movements_map[side_map_tile]
            side_map_tile_position = (position[0] + hor_ver_movement[0], position[1] + hor_ver_movement[1])
            if side_map_tile_position in visited_positions:
                if edge_exists_in_map(plant_map, side_map_tile_position, edge_side):
                    return False
        print(f"\t\tAdding new side {edge_side} at {position}")
        return True
    return False

def count_sides(edge_list):
    sides_count = 0
    #print(edge_list)
    while len(edge_list) > 0:
        # For each edge we remove all edges that are aligned and consecutive with it to count 1 side
        edge = edge_list.pop()
        #print(f"Edge being processed: {edge}")
        edge_side = edge[1]
        initial_position = edge[0]
        for side_direction in side_map_tiles[edge_side]:
            #print(f"\tRemoving edges in the {edge_side} direction and going in the {side_direction} direction")
            movement = hor_ver_movements_map[side_direction]
            next_position = (initial_position[0] + movement[0], initial_position[1] + movement[1])
            while len(edge_list) > 0:
                #print(f"\tEvaluating {next_position}")
                if (next_position, edge_side) in edge_list:
                    #print(f"\t\tRemoving edge {next_position} in the {edge_side} direction")
                    edge_list.remove((next_position, edge_side))
                    next_position = (next_position[0] + movement[0], next_position[1] + movement[1])
                else:
                    break
        sides_count += 1
    return sides_count

def get_plant_single_region_cost_2(plant_map, plant_position):
    area = 0
    sides_count = 0
    plant_position_list = set([plant_position])
    plant_positions_visited = set()
    edge_list = set()
    while len(plant_position_list) > 0:
        position = plant_position_list.pop()
        plant_positions_visited.add(position)
        if plant_map[position[0], position[1]] == '1':
            area += 1
            for hor_ver_movement_key, hor_ver_movement in hor_ver_movements_map.items():
                #sides_count += int(add_new_side(plant_map, position, hor_ver_movement_key, plant_positions_visited))
                if edge_exists_in_map(plant_map, position, hor_ver_movement_key):
                    edge_list.add((position, hor_ver_movement_key))
                next_position = (position[0] + hor_ver_movement[0], position[1] + hor_ver_movement[1])
                # Invalid positions -- out of the map
                if next_position[0] < 0 or next_position[1] < 0:
                    continue
                try:
                    if plant_map[next_position] == '1':
                        if next_position not in plant_positions_visited:
                            plant_position_list.add(next_position)
                except IndexError:
                    pass
    sides_count = count_sides(edge_list)
    #print(f"\tArea: {area} Sides: {sides_count}")

    return area*sides_count, plant_positions_visited

def get_plant_total_fence_cost_2(garden_map, plant_type):
    plant_map, plant_position_list = get_plant_land_in_garden(garden_map, plant_type)
    plant_fence_cost = 0
    while len(plant_position_list) > 0:
        plant_position = plant_position_list.pop()
        added_cost, plant_positions_visited = get_plant_single_region_cost_2(plant_map, plant_position)
        plant_position_list = plant_position_list.difference(plant_positions_visited)
        plant_fence_cost += added_cost
    return plant_fence_cost

def get_all_plants_total_fence_cost_2(garden_map):
    plant_fence_cost = 0
    plant_types = get_all_plant_types_in_garden(garden_map)
    for plant_type in plant_types:
        new_cost = get_plant_total_fence_cost_2(garden_map, plant_type)
        print(f"Plant {plant_type} cost: {new_cost}")
        plant_fence_cost += new_cost
    return plant_fence_cost


In [31]:
# Solve part 2
get_all_plants_total_fence_cost_2(garden_map)

Plant E cost: 34482
Plant H cost: 40224
Plant O cost: 52496
Plant V cost: 37942
Plant R cost: 10816
Plant G cost: 39290
Plant X cost: 31512
Plant C cost: 18786
Plant D cost: 11122
Plant J cost: 32864
Plant B cost: 32216
Plant L cost: 43006
Plant U cost: 31298
Plant N cost: 87378
Plant M cost: 25006
Plant Y cost: 49914
Plant A cost: 30856
Plant S cost: 34142
Plant Q cost: 23538
Plant K cost: 35018
Plant F cost: 17714
Plant W cost: 22086
Plant T cost: 34892
Plant I cost: 24418
Plant Z cost: 65124
Plant P cost: 43926


910066